In [36]:
import pandas as pd
import os
import numpy as np

In [37]:
# in pub pcr events, e situation 13 = initial patient acuity, e disposition 23 = hospital capability

In [38]:
folder = r'C:\Users\gratt\Desktop\emsData\processedCSVs\big\pub_pcrevents_chunks'

In [39]:
filepaths = [os.path.join(folder, filename) for filename in os.listdir(folder)]
filepaths

['C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk1.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk10.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk2.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk3.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk4.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk5.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk6.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk7.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk8.csv',
 'C:\\Users\\gratt\\Desktop\\emsData

In [40]:
def importvars(filepath):
    return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])

In [41]:
df = pd.concat(map(importvars, filepaths))

C:\Users\gratt\AppData\Local\Temp\ipykernel_20884\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_20884\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_20884\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_20884\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read

In [42]:
df

,PcrKey,eDisposition_23,eSituation_02,eSituation_13
0,1472192,7701003,7701003,2813005
1,13273412,9908007,9922001,2813005
2,14548133,7701003,7701001,2813001
3,17184298,7701003,9922005,2813001
4,21699657,7701003,9922001,2813005
...,...,...,...,...
4999995,168637681,7701001,7701003,7701001
4999996,168637682,9908007,9922001,7701003
4999997,168637683,7701003,9922001,2813005
4999998,168637684,9908007,9922001,2813005


In [43]:
# filter for only patients where possible ionjury = yes
df = df[ (df['eSituation_02'] == 9922005)]

In [44]:
# eSituation_13 == initial patient acuity
# eDisposition_23 = hospital capability
df = df.rename(columns = {'eDisposition_23':'hospCapability', 'eSituation_13':'initialPtAcuity'})

In [45]:
df

,PcrKey,hospCapability,eSituation_02,initialPtAcuity
3,17184298,7701003,9922005,2813001
14,102488422,7701003,9922005,2813001
17,113861204,7701003,9922005,2813003
18,116400273,7701003,9922005,2813003
27,118170842,7701001,9922005,7701003
...,...,...,...,...
4999924,168637610,7701003,9922005,7701003
4999943,168637629,7701003,9922005,2813005
4999954,168637640,7701003,9922005,2813005
4999974,168637660,9908007,9922005,7701003


initial patient acuity code list
2813001 - Critical (Red)
2813003 - Emergent (Yellow)
2813005 - Lower Acuity (Green)
2813007 - Dead without Resuscitation Efforts (Black) 
7701001 - NA
7701003 - not recorded

In [46]:
# new codes
# critical == 4
df = df.replace([2813001], 4)
# emergent == 3
df = df.replace([2813003], 3)
# lower acuity == 3
df = df.replace([2813002], 2)
# dead = 1
df = df.replace([2813007], 1)

In [47]:
# throw out missing data
df = df[ (df['initialPtAcuity'] == 1) | (df['initialPtAcuity'] == 2) | (df['initialPtAcuity'] == 3) | (df['initialPtAcuity'] == 4)]

In [48]:
df

,PcrKey,hospCapability,eSituation_02,initialPtAcuity
3,17184298,7701003,9922005,4
14,102488422,7701003,9922005,4
17,113861204,7701003,9922005,3
18,116400273,7701003,9922005,3
33,118170860,9908007,9922005,3
...,...,...,...,...
4999742,168637427,9908007,9922005,3
4999767,168637452,9908007,9922005,3
4999898,168637583,7701003,9922005,3
4999905,168637590,9908007,9922005,3


In [49]:
# throw out missing data for hosp capability
df = df[ (df['hospCapability'] != 7701003) & (df['hospCapability'] != 7701001)]

In [50]:
df

,PcrKey,hospCapability,eSituation_02,initialPtAcuity
33,118170860,9908007,9922005,3
70,118170980,9908007,9922005,3
134,118171192,9908007,9922005,3
145,118171228,9908007,9922005,3
177,118171324,9908007,9922005,4
...,...,...,...,...
4999570,168637255,9908007,9922005,3
4999742,168637427,9908007,9922005,3
4999767,168637452,9908007,9922005,3
4999905,168637590,9908007,9922005,3


In [52]:
# codes for trauma centers
tmaCnts = [9908021, 9908023, 9908025, 9908027, 9908029]

In [53]:
# codes for non trauma centers
notTma = [9908001, 9908003, 9908005, 9908007, 9908009, 9908011, 9908017, 9908019, 9908031, 9908033, 9908035]

In [55]:
# replace all vals for non trauma centers with zeroes
df = df.replace(notTma, value=[0]*11)

In [57]:
# drop the injury column
df = df.drop(columns="eSituation_02")

In [59]:
# replace all vals for trauma center with 1
df = df.replace(tmaCnts, value=[1]*5)

In [60]:
df

,PcrKey,hospCapability,initialPtAcuity
33,118170860,0,3
70,118170980,0,3
134,118171192,0,3
145,118171228,0,3
177,118171324,0,4
...,...,...,...
4999570,168637255,0,3
4999742,168637427,0,3
4999767,168637452,0,3
4999905,168637590,0,3


In [61]:
df.head(25)

,PcrKey,hospCapability,initialPtAcuity
33,118170860,0,3
70,118170980,0,3
134,118171192,0,3
145,118171228,0,3
177,118171324,0,4
182,118171353,0,3
224,118171484,0,3
272,118171640,0,3
331,118171812,0,3
379,118171963,1,4


In [62]:
# now join with socioeoncomic
raceAgeGender = pd.read_csv(r"C:\Users\gratt\Desktop\emsData\raceAgeGender.csv")

In [63]:
raceAgeGender

,PcrKey,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,gender
0,13273412,0,1,0,0,0,0,78,0.0
1,83733589,0,0,0,0,1,0,31,0.0
2,88178354,0,1,0,0,0,0,85,0.0
3,97455081,0,1,0,0,0,0,77,0.0
4,102488422,0,1,0,0,0,0,4,0.0
...,...,...,...,...,...,...,...,...,...
32417152,182540290,1,0,0,0,0,0,77,0.0
32417153,182540377,1,0,0,0,0,0,54,1.0
32417154,182540383,1,0,0,0,0,0,78,0.0
32417155,182541657,0,1,0,0,0,0,74,1.0


In [64]:
df = df.set_index('PcrKey')

In [65]:
df

,hospCapability,initialPtAcuity
PcrKey,,
118170860,0,3
118170980,0,3
118171192,0,3
118171228,0,3
118171324,0,4
...,...,...
168637255,0,3
168637427,0,3
168637452,0,3


In [66]:
raceAgeGender = raceAgeGender.set_index('PcrKey')

In [67]:
raceAgeGender

,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,gender
PcrKey,,,,,,,,
13273412,0,1,0,0,0,0,78,0.0
83733589,0,0,0,0,1,0,31,0.0
88178354,0,1,0,0,0,0,85,0.0
97455081,0,1,0,0,0,0,77,0.0
102488422,0,1,0,0,0,0,4,0.0
...,...,...,...,...,...,...,...,...
182540290,1,0,0,0,0,0,77,0.0
182540377,1,0,0,0,0,0,54,1.0
182540383,1,0,0,0,0,0,78,0.0


In [68]:
df = df.join(raceAgeGender)

In [69]:
df

,hospCapability,initialPtAcuity,black,white,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,ageinyear,gender
PcrKey,,,,,,,,,,
118170860,0,3,0.0,1.0,0.0,0.0,0.0,0.0,49.0,1.0
118170980,0,3,0.0,1.0,0.0,0.0,0.0,0.0,30.0,0.0
118171192,0,3,1.0,0.0,0.0,0.0,0.0,0.0,58.0,1.0
118171228,0,3,0.0,1.0,0.0,0.0,0.0,0.0,78.0,1.0
118171324,0,4,1.0,0.0,0.0,0.0,0.0,0.0,19.0,1.0
...,...,...,...,...,...,...,...,...,...,...
168637255,0,3,0.0,1.0,0.0,0.0,0.0,0.0,60.0,0.0
168637427,0,3,0.0,1.0,0.0,0.0,0.0,0.0,23.0,0.0
168637452,0,3,0.0,1.0,0.0,0.0,0.0,0.0,15.0,0.0


In [70]:
# looks good export
df.to_csv(r"C:\Users\gratt\Desktop\emsData\raceAgeGenderAcuityTrauma.csv")